# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [53]:
# To load in data file "city data all.csv" from the first exercise WeatherPy.

file_to_load = "../WeatherPy/output_data/city_data_all.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df_all = pd.DataFrame(city_data_df)
city_data_df_all.head()

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates
0,Arlit,2447513,NE,7.3853,18.7369,12.26,89.26,4,0,1621982659
1,Hermanus,3366880,ZA,19.2345,-34.4187,7.31,57.90,74,51,1621982581
2,Hendek,745664,TR,30.7481,40.7994,1.95,50.32,81,3,1621982659
3,Dongsheng,1812256,CN,109.9776,39.8161,15.08,54.79,48,39,1621982659
4,Bathsheba,3374083,BB,-59.5167,13.2167,12.57,81.21,61,20,1621982595


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [54]:
# Configure gmaps api for city Lat and Lon and City Humidity

gmaps.configure(api_key=g_key)
city_locations_all_location = city_data_df_all[["City Latitude", "City Longitude"]]
city_locations_all_humidity = city_data_df_all["City Humidity"].astype(float)

In [55]:
# Plot Heatmap

fig_1 = gmaps.figure(map_type="TERRAIN")
# Create heat layer

heat_layer = gmaps.heatmap_layer(city_locations_all_location, weights=city_locations_all_humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)
# Add layer
fig_1.add_layer(heat_layer)

# Display figure
fig_1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
# To narrow down the "city_data" Dataframe to 70-80 degress, 0 cloudiness, and Wind Speed less than 10 mph.

future_vac_sites = (city_data_df_all.loc[city_data_df_all["City Max Temperature"] >= 70])
future_vac_sites_temp = (future_vac_sites.loc[future_vac_sites["City Max Temperature"] <= 80])
future_vac_sites_temp_cloudiness = (future_vac_sites_temp.loc[future_vac_sites_temp["City Cloudiness"] == 0])
future_vac_sites_temp_cloudiness_wind = (future_vac_sites_temp_cloudiness.loc[future_vac_sites_temp_cloudiness["City Wind Speed"] < 10])
future_vac_sites_temp_cloudiness_wind_humid = (future_vac_sites_temp_cloudiness_wind.loc[future_vac_sites_temp_cloudiness_wind["City Humidity"] < 80])
future_vac_sites_temp_cloudiness_wind_humid

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
# To add a new blank column "Hotel Name" for the 6 cities which meet the above crieria.   

hotel_df = future_vac_sites_temp_cloudiness_wind_humid
hotel_df = hotel_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates,Hotel Name
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749,
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752,
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786,
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804,
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806,
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816,


In [58]:
# To use google places to retrieve nearby hotels (for the new Hotel Name column) for the 6 cities which meet 
# the above crieria (70-80 degress,# 0% cloudiness, and Wind Speed less than 10 mph)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    params["location"] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
#     print(json.dumps(response, indent=4, sort_keys=True))
    results = response["results"]
    try:
#     print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("NA")
hotel_df

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates,Hotel Name
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749,بورتو بردية
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752,OlusHome
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786,Birjand Tourist Hotel
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804,Qasr Alzaytuna Inn
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806,Hotel Chik Chik Namibe
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816,Lindsey Hôtel


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [60]:
# Plot Heatmap

gmaps.configure(api_key=g_key)
city_locations_all_location = city_data_df_all[["City Latitude", "City Longitude"]]
city_locations_all_humidity = city_data_df_all["City Humidity"].astype(float)
fig_1 = gmaps.figure(map_type="TERRAIN")

# To create heat layer (humidity for locations)

heat_layer = gmaps.heatmap_layer(city_locations_all_location, weights=city_locations_all_humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)

# Add layer for Humidity for all locations

fig_1.add_layer(heat_layer)

# Add marker layer ontop of heat map for the 6 hotels listed above

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_1.add_layer(markers)

# Display figure for combined heat map and the 6 hotels for ideal weather.  

fig_1

Figure(layout=FigureLayout(height='420px'))